In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
colang_content = """
define user express greeting
  "hello"
  "hi"
  "whats up"

define bot express greeting
  "Hello there!! Can I help you today?"

define flow hello
  user express greeting
  bot express greeting
"""
yaml_content = """
models:
- type: main
  engine: openai
  model: gpt-3.5-turbo-1106
# - type: embeddings
#   engine: openai
#   model: text-embedding-ada-002
"""

from nemoguardrails import LLMRails, RailsConfig

config = RailsConfig.from_content(
  	yaml_content=yaml_content,
    colang_content=colang_content
)

/Users/pragalvhasharma/Downloads/Prag GO to Documents/Comp Sci/MY Projects/NEMO Guardrails DEMO/env/lib/python3.10/site-packages/nemoguardrails/rails/llm/config.py:527: RuntimeWarning: fields may not start with an underscore, ignoring "_type"
  class RailsConfig(BaseModel):
/Users/pragalvhasharma/Downloads/Prag GO to Documents/Comp Sci/MY Projects/NEMO Guardrails DEMO/env/lib/python3.10/site-packages/nemoguardrails/rails/llm/config.py:527: RuntimeWarning: fields may not start with an underscore, ignoring "_source"
  class RailsConfig(BaseModel):


In [3]:
rails = LLMRails(config=config)

AttributeError: module 'openai' has no attribute 'OpenAI'

In [ ]:
res = await rails.generate_async(prompt="hello")
print(res)

NameError: name 'rails' is not defined

In [ ]:
# Passing a complete conversation

messages = [
    {"role": "user", "content": "Hey there!"}
]
res = await rails.generate_async(messages=messages)
print(res)

#Await is only for a .ipynd file

Error in LoggingCallbackHandler.on_chat_model_start callback: TypeError('can only concatenate list (not "str") to list')


{'role': 'assistant', 'content': 'Hello there!! Can I help you today?'}


In [ ]:
#Colang if else statement implemenation
colang_content = """
define user express greeting
    "hello"
    "hi"

define bot express greeting
    "Hello there!! Can I help you today?"

define bot personal greeting
    "Hello $username, nice to see you again!"

define flow hello
    user express greeting
    if $username
        bot personal greeting
    else
        bot express greeting
"""

config = RailsConfig.from_content(
  	yaml_content=yaml_content,
    colang_content=colang_content
)
rails = LLMRails(config=config)

In [ ]:
# When username is not given
messages = [
    {"role": "user", "content": "Hey there!"}
]
res = await rails.generate_async(messages=messages)
print(res)

Error in LoggingCallbackHandler.on_chat_model_start callback: TypeError('can only concatenate list (not "str") to list')


{'role': 'assistant', 'content': 'Hello there!! Can I help you today?'}


In [ ]:
# When given username
messages = [
    {"role": "context", "content": {"username": "Prag"}},
    {"role": "user", "content": "Hey there!"},
]
res = await rails.generate_async(messages=messages)
print(res)

Error in LoggingCallbackHandler.on_chat_model_start callback: TypeError('can only concatenate list (not "str") to list')


{'role': 'assistant', 'content': 'Hello Prag, nice to see you again!'}


LANGCHAIN

In [ ]:
# Langchain integration 
colang_content = """
define user express greeting
    "hello"
    "hi"

define bot express greeting
    "Hello there!! Can I help you today?"

define bot personal greeting
    "Hello $username, nice to see you again!"

define flow hello
    user express greeting
    if $username
      bot personal greeting
    else
        bot express greeting

define flow
    user ...
    $answer = execute qa_chain(query=$last_user_message)
    bot $answer
"""
config = RailsConfig.from_content(
  	yaml_content=yaml_content,
    colang_content=colang_content
)
rails = LLMRails(config=config)

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

vectorstore = Chroma.from_texts(["CatBank's Furry Paws Credit Card offers 2% cashback on pet stores and 1% on all other purchases.",
                                 "Meow Loans at CatBank come with a competitive 5% APR, subject to your catnip credit score.",
                                 "Earn up to 4% interest with CatBank's Kitty Savers account, with a minimum deposit of $500.",
                                 "Invest in the Whisker Growth Fund with a minimum of $1,000 and enjoy purr-tential annual returns of 7%.",
                                 "Open a Feline Secure savings account with just $100; enjoy up to 3% interest with no monthly fees."],
                                embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()
prompt_template = """You are a helpful bot for our Bank. Only answer if your have got content in "Context".
Otherwise tell the user in a friendly way that you do not know and can not help with that.

Context: {context}

Question: {question}
Answer here:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(
    llm=rails.llm, # Use llm attached to rail instance
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
)

AttributeError: module 'openai' has no attribute 'error'

In [ ]:
rails.register_action(qa, name="qa_chain")

In [ ]:
history = [{"role": "user","content": "Whats the minimum deposit for CatBank's Kitty Savers account?"}]
bot_message = await rails.generate_async(messages=history)
print(bot_message)

Error in LoggingCallbackHandler.on_chat_model_start callback: TypeError('can only concatenate list (not "str") to list')
/Users/pragalvhasharma/Downloads/Prag GO to Documents/Comp Sci/MY Projects/NEMO Guardrails DEMO/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `arun` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use ainvoke instead.
  warn_deprecated(
Error in LoggingCallbackHandler.on_chat_model_start callback: TypeError('can only concatenate list (not "str") to list')


{'role': 'assistant', 'content': "The minimum deposit for CatBank's Kitty Savers account is $500."}


In [ ]:
history = [{"role": "user","content": "Make a joke about a cow"}]
bot_message = await rails.generate_async(messages=history)
print(bot_message)

Error in LoggingCallbackHandler.on_chat_model_start callback: TypeError('can only concatenate list (not "str") to list')
Error in LoggingCallbackHandler.on_chat_model_start callback: TypeError('can only concatenate list (not "str") to list')


{'role': 'assistant', 'content': 'Sorry, I can\'t help with that. But I\'m sure cows are "moo-tivated" to have a good laugh!'}


In [ ]:
new_colang_content = """
define user express greeting
    "hello"
    "hi"

define bot express greeting
    "Hello there!! Can I help you with your cat-related banking today?"

define bot personal greeting
    "Hello $username, nice to see you again! Hope your cat's doing well!"

define flow hello
    user express greeting
    if $username
        bot personal greeting
    else
        bot express greeting

define user ask about dogs
    "Can I get a loan for my dog?"
    "Do you provide pet insurance for dogs?"
    "What about doggie credit cards?"

define bot no dogs policy
    "Here at CatBank, we're all about cats! We don't offer services for dogs, but we can help with any cat-related banking needs."

define flow dog policy
    user ask about dogs
    bot no dogs policy

define user silly cat question
    "Can my cat open its own bank account?"
    "Do you think cats are better bankers than humans?"
    "Can I use a picture of my cat as my credit card design?"

define bot respond to silly cat question
    "While we love your cat's enthusiasm, only humans can open bank accounts. But we totally agree that cats would make amazing bankers, and yes, your credit card can definitely feature your cat's majestic portrait!"

define flow silly cat questions
    user silly cat question
    bot respond to silly cat question

define user chit chat
    "What do you think about the latest movie?"
    "Got any weekend plans?"
    "Can you tell me a joke?"

define bot finance only
    "Sorry, I do cat related finance advice only!"

define flow chit chat
    user chit chat
    bot finance only

define flow
    user ...
    $answer = execute qa_chain(query=$last_user_message)
    bot $answer
"""

In [ ]:
config = RailsConfig.from_content(
  	yaml_content=yaml_content,
    colang_content=new_colang_content
)
rails = LLMRails(config=config)
rails.register_action(qa, name="qa_chain")

In [ ]:
messages = [
    {"role": "context", "content": {"username": "Markus"}},
    {"role": "user", "content": "Hello!"},
]
res = await rails.generate_async(messages=messages)
print(res)

Error in LoggingCallbackHandler.on_chat_model_start callback: TypeError('can only concatenate list (not "str") to list')


{'role': 'assistant', 'content': "Hello Markus, nice to see you again! Hope your cat's doing well!"}


In [ ]:
messages = [{"role": "user","content": "Can you tell me a joke?"}]
bot_message = await rails.generate_async(messages=messages)
print(bot_message)

Error in LoggingCallbackHandler.on_chat_model_start callback: TypeError('can only concatenate list (not "str") to list')


{'role': 'assistant', 'content': 'Sorry, I do cat related finance advice only!'}


In [ ]:
from langchain.document_transformers import DoctranPropertyExtractor
from typing import Sequence, Any
from langchain.schema import Document

class CustomDoctranPropertyExtractor(DoctranPropertyExtractor):
    def transform_documents(
            self, documents: Sequence[Document], **kwargs: Any
    ) -> Sequence[Document]:

        """Async method is currently broken (Document cant be used in async)."""
        try:
            from doctran import Doctran, ExtractProperty

            doctran = Doctran(
                openai_api_key=self.openai_api_key, openai_model=self.openai_api_model
            )
        except ImportError:
            raise ImportError(
                "Install doctran to use this parser. (pip install doctran)"
            )

        properties = [ExtractProperty(**property) for property in self.properties]
        for d in documents:
            doctran_doc = doctran.parse(content=d.page_content)
            extracted_properties = doctran_doc.extract(
                properties=properties).execute()

            d.metadata["extracted_properties"] = extracted_properties
        return documents

from langchain.schema import Document

async def get_bank_branch_address(branch):
    properties = [
        {
            "name": "mentioned_branch",
            "description": "The bank branch mentioned in this query.",
            "type": "string",
            "required": True
        }
    ]
    property_extractor = CustomDoctranPropertyExtractor(
        properties=properties, openai_api_model="gpt-3.5-turbo"
    )
    documents = [Document(page_content=branch)]
    extracted_documents = property_extractor.transform_documents(
        documents=documents
    )
    print("EXTRACTED_DOCS: ", extracted_documents)
    branch_name = extracted_documents[0].metadata['extracted_properties'].transformed_content

    branch_addresses = {
        "Central CatBank": "123 Feline Street, Cat City",
        "CatBank North": "456 Purr Avenue, Kitty Corner",
        "CatBank South": "789 Whisker Way, Tabby Town",
        "Downtown CatBank": "101 Claw Circle, Meow Metro"
    }

    address = branch_addresses.get(branch_name)
    if address:
        return f"The address for {branch_name} is {address}."
    else:
        return "Branch not found."



function_colang_content = """
define user express greeting
    "hello"
    "hi"

define bot express greeting
    "Hello there!! Can I help you with your cat-related banking today?"

define bot personal greeting
    "Hello $username, nice to see you again! Hope your cat's doing well!"

define flow hello
    user express greeting
    if $username
        bot personal greeting
    else
        bot express greeting

define user ask about dogs
    "Can I get a loan for my dog?"
    "Do you provide pet insurance for dogs?"
    "What about doggie credit cards?"

define bot no dogs policy
    "Here at CatBank, we're all about cats! We don't offer services for dogs, but we can help with any cat-related banking needs."

define flow dog policy
    user ask about dogs
    bot no dogs policy

define user silly cat question
    "Can my cat open its own bank account?"
    "Do you think cats are better bankers than humans?"
    "Can I use a picture of my cat as my credit card design?"

define bot respond to silly cat question
    "While we love your cat's enthusiasm, only humans can open bank accounts. But we totally agree that cats would make amazing bankers, and yes, your credit card can definitely feature your cat's majestic portrait!"

define flow silly cat questions
    user silly cat question
    bot respond to silly cat question

define user chit chat
    "What do you think about the latest movie?"
    "Got any weekend plans?"
    "Can you tell me a joke?"

define bot finance only
    "Sorry, I do cat related finance advice only!"

define flow chit chat
    user chit chat
    bot finance only

define user inquire branch address
    "Wo finde ich die Central CatBank?"
    "Wo ist CatBank North?"
    "Kannst du mir sagen, wo CatBank South ist?"
    "Wo ist Downtown CatBank?"

define flow inquire branch address
    user inquire branch address
    $address = execute get_bank_branch_address(branch=$last_user_message)
    bot $address

define flow
    user ...
    $answer = execute qa_chain(query=$last_user_message)
    bot $answer
"""

In [ ]:
config = RailsConfig.from_content(
  	yaml_content=yaml_content,
    colang_content=function_colang_content
)
rails = LLMRails(config=config)
rails.register_action(qa, name="qa_chain")
rails.register_action(get_bank_branch_address, name="get_bank_branch_address")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
messages = [{"role": "user","content": "Wo ist die Central CatBank"}]
bot_message = await rails.generate_async(messages=messages)
print(bot_message)

Error in LoggingCallbackHandler.on_chat_model_start callback: TypeError('can only concatenate list (not "str") to list')
Error while execution get_bank_branch_address: 1 validation error for DoctranConfig
openai_deployment_id
  Field required [type=missing, input_value={'openai_model': 'gpt-3.5...enai_token_limit': 8000}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing


{'role': 'assistant', 'content': "I'm sorry, an internal error has occurred."}
